# 第一章：机器学习基础
## 监督学习
监督学习是指：利用一组已知类别的样本调整分类器的参数，使其达到所要求性能的过程，也称为监督训练或有教师学习。
监督学习是从标记的训练数据来推断一个功能的机器学习任务。训练数据包括一套训练示例。在监督学习中，每个实例都是由一个输入对象（通常为矢量）和一个期望的输出值（也称为监督信号）组成。监督学习算法是分析该训练数据，并产生一个推断的功能，其可以用于映射出新的实例。一个最佳的方案将允许该算法来正确地决定那些看不见的实例的类标签。这就要求学习算法是在一种“合理”的方式从一种从训练数据到看不见的情况下形成。

## 非监督学习
在机器学习，无监督学习的问题是，在未加标签的数据中，试图找到隐藏的结构。因为提供给学习者的实例是未标记的，因此没有错误或报酬信号来评估潜在的解决方案。这区别于监督学习和强化学习无监督学习。
无监督学习是密切相关的统计数据密度估计的问题。然而无监督学习还包括寻求，总结和解释数据的主要特点等诸多技术。在无监督学习使用的许多方法是基于用于处理数据的数据挖掘方法。

## 其他资源
### 数学基础
- 知乎上的机器学习涉及的数学基础贴：https://zhuanlan.zhihu.com/p/25197792
- B站上的线性代数教学贴：https://www.bilibili.com/video/av5987715?from=search&seid=5057923081426827439
- 一个还没看的知乎live：https://www.zhihu.com/lives/926880054670159872
- 机器学习资源：https://github.com/apachecn/AiLearning



# 第二章：K-邻近算法
## 后续需要跟进的问题
- python 迭代器
- python 字典操作
- python numpy向量计算

In [58]:
import numpy as np
import operator

def createDataSet():
    group = array([[1.0,1.1],[1.0,1.0],[0,0],[0,0.1]])
    labels = ['A','A','B','B']
    return group,labels

In [59]:
group,labels = createDataSet()
print group
print labels

[[1.  1.1]
 [1.  1. ]
 [0.  0. ]
 [0.  0.1]]
['A', 'A', 'B', 'B']


In [60]:
group.shape[0]
group.sum(axis=0)

array([2. , 2.2])

In [61]:
def classify0(inX, dataSet, labels, k):
    dataSetSize = dataSet.shape[0]
    diffMat = tile(inX, (dataSetSize, 1)) - dataSet
    sqDiffMat = diffMat**2
    sqDistances = sqDiffMat.sum(axis=1)
    distances = sqDistances**0.5
    sorteDistIndicies = distances.argsort()
    #print sorteDistIndicies
    classCount = {}
    for i in range(k):
        voteIlabel = labels[sorteDistIndicies[i]]
        #print ''
        #print voteIlabel
        classCount[voteIlabel] = classCount.get(voteIlabel, 0) + 1 
        #这里添加相当于以分类为key，以分类出现的次数为key的值，就是一个投票。py可以用这种方式进行字典的初始化。
        #print classCount[voteIlabel]
        #print 'classCount:' 
        #print  classCount
    sortedClassCount = sorted(classCount.iteritems(),key=operator.itemgetter(1),reverse=True)
    # itemgetter(1)是按照key位置为1的进行排序，这里就是按照分类得票数量进行排序，并且逆序，从大到小排序。
    #print 'final'
    #print classCount
    #print sortedClassCount
    
    return sortedClassCount[0][0]

In [62]:
classify0([0,0], group, labels, 3)

'B'

In [92]:
def classify01(inX, dataSet, labels, k):
    dist = np.sum((inX - dataSet)**2, axis=1)**0.5
    k_labels = [labels[index] for index in dist.argsort()[0 : k]]
    print k_labels
    label = Counter(k_labels).most_common(1)[0][0]
    #print Counter(k_labels).most_common() most_common中的参数是指的现实几个
    return label

In [88]:
classify01([0,0], group, labels, 3)

['B', 'B', 'A']
[('B', 2), ('A', 1)]


'B'

In [91]:
newDic = {'A':1,'B':1,'C':1,'D':1,'E':1,'A':1,'A':1}
Counter(newDic).most_common(1)

[('A', 1)]